# Instalacion de dependenciasv (ONLY COLLAB)

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

# Instalacion de dependencias

In [ ]:
import os
import warnings

!pip install --upgrade pip

!pip uninstall unsloth torch torchvision torchaudio xformers -y
!pip cache purge

!pip install torch==2.4.* torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

!pip install "unsloth[cu124-ampere-torch240] @ git+https://github.com/unslothai/unsloth.git"

# Unsloth

In [1]:
pip install --no-deps --upgrade transformers==4.51.3 trl==0.15.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show unsloth-zoo

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip uninstall unsloth==2025.5.8 unsloth-zoo

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install unsloth==2025.5.7 unsloth-zoo==2025.5.8

  Using cached unsloth-2025.5.7-py3-none-any.whl.metadata (47 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 182.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 223.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 265.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 265.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 303.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Atte

In [ ]:
!pip install vllm

In [ ]:
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer wandb

In [5]:
#pip install --upgrade pillow

Note: you may need to restart the kernel to use updated packages.


# GEMMMA 3

In [54]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.5.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.9.0.1.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.403 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


### We now add LoRA adapters so we only need to update a small amount of parameters!

In [55]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 32, #8          # Larger = higher accuracy, but might overfit
    lora_alpha = 32, #8  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


# Preparacion de datos

In [56]:
from datasets import load_dataset
dataset = load_dataset("somosnlp-hackathon-2025/ibero-tales-es", split = "train")
dataset

Dataset({
    features: ['id', 'nombre', 'pais', 'tema', 'trama', 'pregunta', 'respuesta'],
    num_rows: 2283
})

In [57]:
# Seleccionar y renombrar las columnas
dataset = dataset.select_columns(['pregunta', 'respuesta']).rename_columns({
    'pregunta': 'question',
    'respuesta': 'answer'
})
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 2283
})

In [58]:
dataset[0]["question"]

"Crea una historia basada en La Difunta Correa, con la trama 'una mujer vaga por las tierras argentinas, ofreciendo consuelo y guía a quienes lo necesitan' y el tema 'libertad y sanación'. ¿Cómo encuentran consuelo aquellos que la encuentran?"

In [59]:
dataset[0]["answer"]

'La Difunta Correa, liberada de su trágica muerte, deambula por las vastas planicies de Argentina. Con su espíritu libre y bondadoso, se convierte en un faro de esperanza para aquellos que se aventuran en las solitarias carreteras.\n\nUna noche, una joven pareja se pierde en su viaje hacia Mendoza. La desesperación los invade, pero entonces aparece La Difunta Correa, con su figura etérea y su dulce sonrisa. Les ofrece agua fresca de un cántaro y les indica el camino de regreso a la seguridad. La pareja, agradecida, escucha sus sabios consejos y encuentra consuelo en su presencia tranquilizadora.\n\nOtros viajeros, abatidos por la tristeza y la pérdida, también encuentran consuelo en sus brazos maternales. La Difunta Correa les brinda palabras de aliento y les ayuda a sanar sus corazones rotos. Con cada alma a la que ayuda, su espíritu se fortalece y su leyenda crece, convirtiéndola en un símbolo de libertad y esperanza para todos los argentinos.'

We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:

In [60]:
# Define las etiquetas para separar razonamiento y solución
reasoning_start = "<think>"
reasoning_end = "</think>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""Resuelve el siguiente problema.  
Primero, piensa en voz alta qué debes hacer, paso por paso y de forma resumida, entre {reasoning_start} y {reasoning_end}.  
Luego, da la respuesta final entre {solution_start} y {solution_end}.  
No escribas nada fuera de ese formato."""
system_prompt

'Resuelve el siguiente problema.  \nPrimero, piensa en voz alta qué debes hacer, paso por paso y de forma resumida, entre <think> y </think>.  \nLuego, da la respuesta final entre <SOLUTION> y </SOLUTION>.  \nNo escribas nada fuera de ese formato.'

Let's map the dataset! and see the first row:

In [61]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": x["answer"],
})
dataset[0]

Map:   0%|          | 0/2283 [00:00<?, ? examples/s]

{'question': "Crea una historia basada en La Difunta Correa, con la trama 'una mujer vaga por las tierras argentinas, ofreciendo consuelo y guía a quienes lo necesitan' y el tema 'libertad y sanación'. ¿Cómo encuentran consuelo aquellos que la encuentran?",
 'answer': 'La Difunta Correa, liberada de su trágica muerte, deambula por las vastas planicies de Argentina. Con su espíritu libre y bondadoso, se convierte en un faro de esperanza para aquellos que se aventuran en las solitarias carreteras.\n\nUna noche, una joven pareja se pierde en su viaje hacia Mendoza. La desesperación los invade, pero entonces aparece La Difunta Correa, con su figura etérea y su dulce sonrisa. Les ofrece agua fresca de un cántaro y les indica el camino de regreso a la seguridad. La pareja, agradecida, escucha sus sabios consejos y encuentra consuelo en su presencia tranquilizadora.\n\nOtros viajeros, abatidos por la tristeza y la pérdida, también encuentran consuelo en sus brazos maternales. La Difunta Corre

We create a regex format to match the reasoning sections and answers:

In [62]:
import re
# Regex para detectar razonamiento y solución juntos
match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [63]:
match_format.search(
    "<think>Déjame pensar como....</think>"\
    "<SOLUTION>Esta historia cuenta como...</SOLUTION>",
)

<re.Match object; span=(0, 86), match='<think>Déjame pensar como....</think><SOLUTION>Es>

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [64]:
# Función que da 3 puntos si el formato completo es correcto
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        if match_format.search(response) is not None:
            score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [65]:
# Función que da puntos parciales si cada etiqueta aparece exactamente una vez
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Contamos cuántas palabras clave se veen. ¡Se penaliza si se veen demasiadas! 
        # Si se vee 1, ¡Adicionamos puntos!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [72]:
# Función para comparar la solución generada con la respuesta esperada usando similitud simple
def check_text_similarity(prompts, completions, answer, **kwargs):
    scores = []
    for completion, true_answer in zip(completions, answer):
        response = completion[0]["content"]
        match = match_format.search(response)
        if not match:
            scores.append(0)
            continue
        generated_answer = match.group(1).strip().lower()
        true_answer = true_answer.strip().lower()

        gen_words = set(generated_answer.split())
        true_words = set(true_answer.split())
        common = gen_words.intersection(true_words)

        if len(true_words) == 0:
            scores.append(0)
            continue

        similarity = len(common) / len(true_words)  # Valor entre 0 y 1
        score = similarity * 3  # Máximo 3 puntos
        scores.append(score)
    return scores

## Configuracion Wandb

In [ ]:
pip install wandb

In [7]:
import os
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/user/.netrc
wandb: Currently logged in as: da-qc to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
wandb_project = "iberotales-gemma-3-1b-it-es"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
ඞ = os.environ["WANDB_PROJECT"]
print(ඞ)

iberotales-gemma-3-1b-it-es


In [67]:
! nvidia-smi

Mon Jun  2 22:49:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    On  |   00000000:30:00.0 Off |                    0 |
| N/A   35C    P0            100W /  350W |    4775MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Pipeline

In [73]:
max_prompt_length = 512

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused", #paged_adamw_8bit
    logging_steps = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = 1024, # max_seq_length - max_prompt_length
    num_train_epochs = 3, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the reward column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

In [74]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_text_similarity,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,283 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 26,091,520/1,025,977,472 (2.54% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_text_similarity
1,0.000000,1.408629,2.790121,633.312500,0.000000,0.750000,0.437500,0.221129
2,0.000000,1.709521,2.485871,663.437500,0.000000,0.750000,0.750000,0.209521
3,0.000000,0.860035,1.313564,687.000000,0.001035,0.187500,0.625000,0.047535
4,0.000000,1.524498,2.676237,646.187500,0.000976,0.750000,0.562500,0.211998
5,0.000000,2.415219,2.523774,717.875000,0.001091,1.125000,0.937500,0.352719
6,0.000000,1.821678,2.271278,660.437500,0.001143,0.750000,0.875000,0.196679
7,0.000000,1.494104,2.576724,659.375000,0.001162,0.750000,0.500000,0.244104
8,0.000000,1.683754,2.571864,603.250000,0.001195,0.750000,0.687500,0.246254
9,0.000000,1.697543,3.088143,783.000000,0.001198,0.937500,0.500000,0.260043
10,0.000100,2.220656,2.901351,726.750000,0.001278,1.125000,0.687500,0.408156


TrainOutput(global_step=50, training_loss=0.00013613314047688618, metrics={'train_runtime': 9811.2194, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.005, 'total_flos': 0.0, 'train_loss': 0.00013613314047688618})

# INFERENCIA

In [85]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Crea una historia corta basada en Wiracocha, con la trama 'Héroe'"},
]

text = tokenizer.apply_chat_template
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<bos><bos><start_of_turn>user
Resuelve el siguiente problema.  
Primero, piensa en voz alta qué debes hacer, paso por paso y de forma resumida, entre <think> y </think>.  
Luego, da la respuesta final entre <SOLUTION> y </SOLUTION>.  
No escribas nada fuera de ese formato.

Crea una historia corta basada en Wiracocha, con la trama 'Héroe<end_of_turn>
<start_of_turn>model
<think>
Okay, el problema es crear una historia corta basada en Wiracocha, enfocándose en el personaje del "Héroe".  Esto implica un equilibrio entre la descripción de Wiracocha y la construcción de un personaje que podamos conectar con.  Necesito una historia con un conflicto y una resolución que sea coherente con su filosofía.  Haré de la historia un relato breve, con un toque de misterio y un final que permita a Wiracocha reflexionar sobre su propio legado. No quiero una historia extensa, solo una narración concisa.
</think>
<SOLUTION>
El río color esmeralda, Viru, serpenteaba entre las montañas, sus aguas silencios

# Guardado o carga modelos alineados

Para guardar el modelo final como adaptadores LoRA, puedes usar `push_to_hub` de Hugging Face para guardarlo en línea o `save_pretrained` para una copia local.

In [76]:
from huggingface_hub import notebook_login
notebook_login()

## Guardar adaptador

**[NOTA]** Esto SOLO guarda los adaptadores LoRA, no el modelo completo. Para guardarlo en 16 bits o en formato GGUF, desplázate hacia abajo.

In [96]:
model.save_pretrained("gemma-3")   # guardar localmente
tokenizer.save_pretrained("gemma-3") # guardar localmente

('gemma-3/tokenizer_config.json',
 'gemma-3/special_tokens_map.json',
 'gemma-3/tokenizer.model',
 'gemma-3/added_tokens.json',
 'gemma-3/tokenizer.json')

In [77]:
model.push_to_hub("daqc/iberotales-gemma-3-1b-it-es-adapter") # guardar Online
tokenizer.push_to_hub("daqc/iberotales-gemma-3-1b-it-es-adapter") # guardar Online 

README.md:   0%|          | 0.00/571 [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/104M [00:00<?, ?B/s]

Saved model to https://huggingface.co/daqc/iberotales-gemma-3-1b-it-es


Uploading...:   0%|          | 0.00/38.1M [00:00<?, ?B/s]

## Guardando en float16 para VLLM
Guuarda directamente en float16 para su despliegue. Se guarda en la carpeta gemma-3-finetune. Cámbialo de False a True para permitir que se ejecute.

In [97]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 1 files from cache to gemma-3-finetune.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:09<00:00,  9.24s/it]


Si quieres subir o hacer push a tu cuenta de Hugging Face, cambia de False a True, agrega tu token de Hugging Face y la ubicación de subida.

In [98]:
if True: # Change to True to upload finetune
    model.push_to_hub_merged("daqc/iberotales-gemma-3-1b-it-es-finetune", tokenizer)

Uploading...:   0%|          | 0.00/38.1M [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 1 files from cache to daqc/iberotales-gemma-3-1b-it-es-finetune.


Unsloth: Merging weights into 16bit:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading...:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:21<00:00, 21.34s/it]


## Conversión a GGUF / llama.cpp

Guarda en `GGUF` / `llama.cpp`. Puedes convertir a precisiones `Q8_0`, `F16` o `BF16`.


In [103]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Unsloth GGUF:hf-to-gguf:Loading model: gemma-3-finetune
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {1152, 262144}
Unsloth GGUF:hf-to-gguf:output_norm.weight,                torch.bfloat16 --> F32, shape = {1152}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting special token type bos to 2
Unsloth GGUF:gguf.vocab:Setting special token type eos to 106
Unsloth GGUF:gguf.vocab:Setting special token type unk to 3
Unsloth GGUF:gguf.vocab:Setting special token type pad to 0
Unsloth GGUF:gguf.vocab:Setting add_bos_token to True
Unsloth G

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to gemma-3-finetune.Q8_0.gguf with size = 1.1G
Unsloth: Successfully saved GGUF to:
gemma-3-finetune.Q8_0.gguf


Likewise, if you want to instead push to GGUF to your Hugging Face account, set if False to if True and add your Hugging Face token and upload location!

In [102]:
if True: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "daqc/iberotales-gemma-3-1b-it-es-finetune-gguf",
    )

Unsloth GGUF:hf-to-gguf:Loading model: gemma-3-finetune
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {1152, 262144}
Unsloth GGUF:hf-to-gguf:output_norm.weight,                torch.bfloat16 --> F32, shape = {1152}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting special token type bos to 2
Unsloth GGUF:gguf.vocab:Setting special token type eos to 106
Unsloth GGUF:gguf.vocab:Setting special token type unk to 3
Unsloth GGUF:gguf.vocab:Setting special token type pad to 0
Unsloth GGUF:gguf.vocab:Setting add_bos_token to True
Unsloth G

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to gemma-3-finetune.Q8_0.gguf with size = 1.1G
Unsloth: Successfully saved GGUF to:
gemma-3-finetune.Q8_0.gguf


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
